# Demo 04: Ray Tracing and Sensitivity Kernels on 1D Spherical Meshes

This notebook demonstrates ray tracing and sensitivity kernel computation using **1D spherical meshes** for radially-symmetric planet models. This is the 1D equivalent of demo 02, showcasing the same workflow but optimized for spherical symmetry.

**Key Features:**
- Generate 1D spherical meshes (faster and simpler than 3D tetrahedral)
- Compute ray path lengths through spherical shells
- Calculate sensitivity kernels for seismic properties
- Visualize radial profiles using `plot_shell_property()`
- Process single and multiple rays with the same unified API

In [ ]:
# Setup: Enable/disable interactive plotting
INTERACTIVE = True  # Set to False for batch processing

import sys
import numpy as np
import matplotlib.pyplot as plt

# Import SensRay
sys.path.insert(0, '../')
from sensray import PlanetModel

print("SensRay loaded successfully!")
print(f"Interactive mode: {INTERACTIVE}")

## 1. Create Planet Model and 1D Spherical Mesh

We'll use the PREM model and create a 1D spherical mesh with major discontinuities.

In [ ]:
# Load PREM model
model = PlanetModel.from_standard_model("prem")
print(f"Loaded model: {model.name}")
print(f"Radius: {model.radius} km")

# Define layer boundaries with variable spacing
# PREM discontinuities: ICB at 1221.5 km, CMB at 3480 km, surface at 6371 km
ICB_radius = 1221.5  # Inner core boundary
CMB_radius = 3480.0  # Core-mantle boundary

# Inner core: uniform spacing (e.g., 100 km)
dr_inner_core = 100.0
radii_inner_core = np.arange(0, ICB_radius, dr_inner_core)

# Outer core: uniform spacing (e.g., 150 km)
dr_outer_core = 150.0
radii_outer_core = np.arange(ICB_radius, CMB_radius, dr_outer_core)

# Mantle: increasing spacing from bottom to top
# Small spacing near CMB (lower mantle), larger spacing near surface (upper mantle)
n_mantle = 20  # Number of layers in mantle
# Create increasing spacing using quadratic progression
spacing_factor = np.linspace(1, 3, n_mantle)  # Increases from 1x to 3x
cumulative_thickness = np.cumsum(spacing_factor)
cumulative_thickness = np.flip(cumulative_thickness) / cumulative_thickness[-1] * (model.radius - CMB_radius)
radii_mantle = CMB_radius + cumulative_thickness

# Combine all radii (in descending order for mesh generation)
radii = np.concatenate([radii_inner_core, radii_outer_core, radii_mantle, [model.radius]])
radii = np.unique(radii)  # Remove any duplicates
radii = radii[::-1]  # Reverse to descending order

print(f"\nRadii spacing:")
print(f"  Inner core: {len(radii_inner_core)} layers, dr = {dr_inner_core} km")
print(f"  Outer core: {len(radii_outer_core)} layers, dr = {dr_outer_core} km")
print(f"  Mantle: {len(radii_mantle)} layers, variable spacing")
print(f"  Total boundaries: {len(radii)}")

print(f"\nGenerating 1D spherical mesh with {len(radii)} boundaries...")
model.mesh.generate_spherical_mesh(radii=radii)
print(f"Created mesh with {model.mesh.mesh.n_cells} layers")

## 2. Populate Mesh with Seismic Properties

Map properties from the continuous model onto the discrete 1D mesh cells.

In [ ]:
# Populate properties using volume-weighted radial integration
model.mesh.populate_properties(['vp', 'vs', 'rho'])

# Verify properties are stored
print("Properties in mesh.cell_data:")
for prop in ['vp', 'vs', 'rho']:
    values = model.mesh.mesh.cell_data[prop]
    print(f"  {prop}: {len(values)} values, range [{values.min():.2f}, {values.max():.2f}]")

## 3. Visualize Properties on 1D Mesh

Use `plot_shell_property()` to visualize radial profiles.

In [ ]:
# Plot velocity profiles
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, prop, title in zip(axes, ['vp', 'vs', 'rho'], ['P-wave Velocity', 'S-wave Velocity', 'Density']):
    radii = model.mesh.mesh.radii
    values = model.mesh.mesh.cell_data[prop]

    # Create step plot
    r_plot = []
    v_plot = []
    for i in range(len(values)):
        r_plot.extend([radii[i], radii[i+1]])
        v_plot.extend([values[i], values[i]])

    ax.plot(r_plot, v_plot, 'b-', linewidth=2)
    ax.set_xlabel('Radius (km)')
    ax.set_ylabel(f'{prop} ({["km/s", "km/s", "g/cm³"][["vp", "vs", "rho"].index(prop)]})')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

    # Add vertical lines at boundaries
    for r in radii:
        ax.axvline(r, color='gray', linestyle='--', alpha=0.3)

plt.tight_layout()
if INTERACTIVE:
    plt.show()
else:
    plt.close()

## 4. Define Source-Receiver Geometry

Set up seismic source and receiver for ray tracing.

In [ ]:
# Source and receiver parameters
source_depth_km = 10.0  # Shallow earthquake
distance_deg = 60.0     # Epicentral distance

print(f"Source depth: {source_depth_km} km")
print(f"Epicentral distance: {distance_deg}°")

## 5. Compute Ray Paths with TauP

Calculate ray paths for P, S, and ScS phases.

In [ ]:
# Compute ray paths for multiple phases
phases = ['P', 'S', 'ScS']
rays_dict = {}

print("Computing ray paths...")
for phase in phases:
    rays = model.taupy_model.get_ray_paths(
        source_depth_in_km=source_depth_km,
        distance_in_degree=distance_deg,
        phase_list=[phase]
    )
    if rays:
        rays_dict[phase] = rays[0]  # Take first arrival
        print(f"  {phase}: {len(rays[0].path)} points, travel time = {rays[0].time:.2f} s")
    else:
        print(f"  {phase}: No rays found")

print(f"\nTotal phases with rays: {len(rays_dict)}")

## 6. Compute Ray Path Lengths Through Layers

Use the consolidated `compute_ray_lengths()` API that works for both 1D and 3D meshes.

In [ ]:
# Compute path lengths for each phase
lengths_dict = {}

for phase, ray in rays_dict.items():
    lengths = model.mesh.compute_ray_lengths(ray, store_as=f'{phase}_lengths')
    lengths_dict[phase] = lengths
    print(f"{phase} path lengths: total = {lengths.sum():.2f} km, "
          f"max in layer = {lengths.max():.2f} km")

## 7. Visualize Ray Path Lengths

Plot radial profiles showing how much each ray passes through each layer.

In [ ]:
# Plot path lengths for all phases
fig, ax = plt.subplots(figsize=(10, 5))

radii = model.mesh.mesh.radii
colors = {'P': 'blue', 'S': 'red', 'ScS': 'green'}

for phase in phases:
    if phase in lengths_dict:
        lengths = lengths_dict[phase]

        # Create step plot
        r_plot = []
        l_plot = []
        for i in range(len(lengths)):
            r_plot.extend([radii[i], radii[i+1]])
            l_plot.extend([lengths[i], lengths[i]])

        ax.plot(r_plot, l_plot, label=f'{phase} wave',
                color=colors.get(phase, 'gray'), linewidth=2)

ax.set_xlabel('Radius (km)')
ax.set_ylabel('Path Length (km)')
ax.set_title('Ray Path Lengths Through Layers')
ax.legend()
ax.grid(True, alpha=0.3)

# Add vertical lines at boundaries
for r in radii:
    ax.axvline(r, color='gray', linestyle='--', alpha=0.2)

plt.tight_layout()
if INTERACTIVE:
    plt.show()
else:
    plt.close()

## 8. Compute Sensitivity Kernels for Single Rays

Calculate sensitivity kernels K = -L/v² for velocity perturbations.

In [ ]:
# Compute sensitivity kernels for P and S waves
kernels_dict = {}

for phase, ray in rays_dict.items():
    if phase in ['P', 'S']:  # P uses vp, S uses vs
        property_name = 'vp' if phase == 'P' else 'vs'
        kernel = model.mesh.compute_sensitivity_kernel(
            ray,
            property_name=property_name,
            attach_name=f'K_{phase}'
        )
        kernels_dict[phase] = kernel
        print(f"{phase} kernel: min = {kernel.min():.3e}, max = {kernel.max():.3e}")

## 9. Visualize Sensitivity Kernels

Plot the kernels to see which layers contribute most to travel time perturbations.

In [ ]:
# Plot sensitivity kernels
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, phase in zip(axes, ['P', 'S']):
    if phase in kernels_dict:
        kernel = kernels_dict[phase]
        radii = model.mesh.mesh.radii

        # Create step plot
        r_plot = []
        k_plot = []
        for i in range(len(kernel)):
            r_plot.extend([radii[i], radii[i+1]])
            k_plot.extend([kernel[i], kernel[i]])

        ax.plot(r_plot, k_plot, color=colors.get(phase, 'gray'), linewidth=2)
        ax.set_xlabel('Radius (km)')
        ax.set_ylabel('Sensitivity Kernel (s/km)')
        ax.set_title(f'{phase}-wave Sensitivity Kernel')
        ax.grid(True, alpha=0.3)

        # Add vertical lines at boundaries
        for r in radii:
            ax.axvline(r, color='gray', linestyle='--', alpha=0.2)

        # Add zero line
        ax.axhline(0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
if INTERACTIVE:
    plt.show()
else:
    plt.close()

## 10. Process Multiple Rays Together

Use the consolidated API to compute kernels for multiple rays at once.

In [ ]:
# Compute multiple P-wave rays at different distances
distances = [30, 60, 90]
p_rays = []

print("Computing P-wave rays at multiple distances:")
for dist in distances:
    rays = model.taupy_model.get_ray_paths(
        source_depth_in_km=source_depth_km,
        distance_in_degree=dist,
        phase_list=['P']
    )
    if rays:
        p_rays.append(rays[0])
        print(f"  {dist}°: {len(rays[0].path)} points, time = {rays[0].time:.2f} s")

print(f"\nTotal P-wave rays: {len(p_rays)}")

## 11. Compute Accumulated Sensitivity Kernel

Sum kernels from multiple rays to see overall sensitivity pattern.

In [ ]:
# Compute summed kernel using accumulate='sum'
K_sum = model.mesh.compute_sensitivity_kernel(
    p_rays,
    property_name='vp',
    attach_name='K_P_sum',
    accumulate='sum'
)

print(f"Accumulated kernel: shape = {K_sum.shape}, "
      f"sum = {K_sum.sum():.3e}, "
      f"range = [{K_sum.min():.3e}, {K_sum.max():.3e}]")

# Also compute individual kernels for comparison
K_individual = model.mesh.compute_sensitivity_kernel(
    p_rays,
    property_name='vp',
    accumulate=None  # Returns 2D array
)

print(f"Individual kernels: shape = {K_individual.shape}")

## 12. Visualize Individual vs Accumulated Kernels

Compare individual ray kernels with their sum.

In [ ]:
# Plot individual and summed kernels
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

radii = model.mesh.mesh.radii

# Left: individual kernels
ax = axes[0]
for i, dist in enumerate(distances):
    kernel = K_individual[i]

    # Create step plot
    r_plot = []
    k_plot = []
    for j in range(len(kernel)):
        r_plot.extend([radii[j], radii[j+1]])
        k_plot.extend([kernel[j], kernel[j]])

    ax.plot(r_plot, k_plot, label=f'{dist}°', linewidth=2, alpha=0.7)

ax.set_xlabel('Radius (km)')
ax.set_ylabel('Sensitivity Kernel (s/km)')
ax.set_title('Individual P-wave Kernels')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)

# Right: accumulated kernel
ax = axes[1]
r_plot = []
k_plot = []
for i in range(len(K_sum)):
    r_plot.extend([radii[i], radii[i+1]])
    k_plot.extend([K_sum[i], K_sum[i]])

ax.plot(r_plot, k_plot, color='darkblue', linewidth=2)
ax.set_xlabel('Radius (km)')
ax.set_ylabel('Sensitivity Kernel (s/km)')
ax.set_title('Accumulated P-wave Kernel (Sum)')
ax.grid(True, alpha=0.3)
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)

# Add boundaries
for ax in axes:
    for r in radii:
        ax.axvline(r, color='gray', linestyle='--', alpha=0.2)

plt.tight_layout()
if INTERACTIVE:
    plt.show()
else:
    plt.close()

## 13. Save Mesh with All Computed Data

Export the mesh with all properties, ray lengths, and kernels.

In [ ]:
# List all computed properties
print("Properties stored in mesh:")
model.mesh.list_properties(show_stats=True, top_n=5)

# Save mesh
output_path = "prem_spherical_mesh_with_rays_kernels"
model.mesh.save(output_path)
print(f"\nMesh saved to {output_path}.*")

## Summary

This notebook demonstrated:

1. **1D Spherical Mesh Generation**: Fast and simple for radially-symmetric models
2. **Property Mapping**: Volume-weighted integration from continuous model to discrete layers
3. **Ray Tracing**: Works identically to 3D meshes using the same API
4. **Path Lengths**: Exact geometric intersection with spherical shells
5. **Sensitivity Kernels**: K = -L/v² computation for single and multiple rays
6. **Consolidated API**: Same methods work for both 1D and 3D meshes
   - `compute_ray_lengths()`: single or multiple rays automatically
   - `compute_sensitivity_kernel()`: with `accumulate` parameter for summing

**Advantages of 1D Spherical Meshes:**
- Much faster than 3D tetrahedral meshes
- Simpler visualization (radial profiles vs 3D cross-sections)
- Perfect for testing and radially-symmetric problems
- Identical API to 3D meshes (seamless workflow)